In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661679 sha256=79c622a383b489f7b892a049bdc1065d4fb4066ba0250f6466f9ab356947581b
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 119.1 MB/s eta 0:00:00
     ━━━━━

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-z741j92o/kobert-tokenizer_32679753b87248049d7db38d084dca2f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-z741j92o/kobert-tokenizer_32679753b87248049d7db38d084dca2f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=082164ec527d4cba7a2162cf5c9b4d970cddcb69d0fb7465f9eebcdadfecfd64
  Stored in directory: /tmp/pip-ephem-wheel-cache-x5do1kvx/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cuda:0")

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

In [9]:
data = pd.read_csv('/content/korean_data.csv')

In [10]:
data['Emotion'].unique()

array(['fear', 'happy', 'neutral', 'surprise', 'sad', 'dislike', 'angry',
       'contempt'], dtype=object)

In [12]:
test_data = pd.read_csv("/content/test_split.csv")

In [13]:
data=test_data

In [14]:
data.loc[(data['Emotion'] == "fear"), 'Emotion'] = 0  # fear = 0
data.loc[(data['Emotion'] == "surprise"), 'Emotion'] = 1  # surprise = 1
data.loc[(data['Emotion'] == "angry"), 'Emotion'] = 2  # angry = 2
data.loc[(data['Emotion'] == "sad"), 'Emotion'] = 3  # sad = 3
data.loc[(data['Emotion'] == "neutral"), 'Emotion'] = 4  # neutral = 4
data.loc[(data['Emotion'] == "happy"), 'Emotion'] = 5  # happy = 5
data.loc[(data['Emotion'] == "dislike"), 'Emotion'] = 6  # dislike = 6
data.loc[(data['Emotion'] == "contempt"), 'Emotion'] = 7 # contempt = 7

In [15]:
data_list = []
for ques, label in zip (data['Text'], data['Emotion']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [16]:
print(data_list[:10])

[['안 그래도 노인들은 무시 당하는 세상인데… 너무 부당한 일이 많잖아.', '4'], ['다음주에 엄마 생일이잖아. 잊은 건 아니지?', '1'], ['우와 그림 잘 그리시나봐요.', '4'], ['할 일이 너무 많은데 채점까지 우리한테 시키니까. 그게 본업도 아니고 한가한 것도 아닌데. 오늘 해야죠 뭐.', '6'], ['미안해, 누나. 이번에는 약속 진짜 지킬게. 책 새로 살 건 아니지?', '3'], ['그래도 정훈이랑 둘, 살기는 나쁘지 않네. 집 커봐야 집안일만 많아.', '4'], ['그게 뭐에요?', '4'], ['그냥 모르는 척 해.', '0'], ['헬스만 몇 년을 했지. 그런데 몰려다니는 아줌마들끼리 싸우기도 하고, 아무튼 좀 매너리즘에 빠져서 그만뒀어.', '6'], ['아유 언니 그런 소리 하지를 마. 이렇게 정정한데, 한참 더 살아야지.', '3']]


In [17]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.3, shuffle = True, random_state = 42)

In [18]:
print(len(dataset_train), len(dataset_test))

16073 6889


In [19]:
dataset_test

[['밖에서 일하고 왔는데 집에서 편하게 찌개도 못먹냐? 내가 너한테 벌어다주는 돈이 얼만데 밥할 때 간 잘하라고 말도 못하냐?', '2'],
 [' 면세점에 있는 것들은 다 비싼 브랜드들이야.\t 명품밖에 없잖아.', '3'],
 ['그래~ 좋아해도 이렇게 선물 사는 건 쉬운 일이 아니지.', '5'],
 ['아빠! 오늘 우리 같이 축구하러 가기로 했잖아! 어? 일어나!', '4'],
 ['네… 2년 간 제 정신 아니었죠.        그래도 교수님이 졸업 시켜 주셔서 다행이에요.', '3'],
 ['네. 바로 다시 하겠습니다. 죄송합니다.', '3'],
 ['할머니하곤 해봤지.', '4'],
 ['오늘 도시락 싸왔지. 나가서 먹으면 비싸잖아.', '3'],
 ['그래서 물밖에 나와서 그 초등학생한테 너 왜 그랬냐고 물어봤지..', '4'],
 ['왔어? 식장엔. 잘 다녀왔고?', '4'],
 ['응 서른 여덟인가?', '4'],
 ['네. 그래서 이것저것 알아보고 있어요.', '4'],
 [' 엄마 성격 진짜 더러웠지. 당신 기분 안 좋으면 우리한테 다 풀고.', '7'],
 [' 뭐... 화장품이랑, 술이랑, 과자랑..  뭐 그런 것들이요.', '4'],
 ['그래. 그, 돌잔치를 어디서 하는데?', '4'],
 ['맛있으니까 맛있다고 하는거죠!', '5'],
 ['아이고, 그런 얘기 그냥 만나서 하면 되지.', '4'],
 ['그래서, 허리가 많이 아프대?', '3'],
 ['내가 필통을 안 가지고 간거야. 그랬더니 예찬이가 나한테          샤프랑 지우개랑 막 빌려줬어.', '3'],
 ['이번에 서울에 새로 아쿠아리움이 문을 열었다고 하던데. 뉴스에서 봤어?', '4'],
 ['수행평가?  공부하는데 뭔 놈의 돈이 이렇게 많이 들어, 맨날.', '1'],
 ['삼천? 아후.. 너무 훅 내리는 거 아니야?', '1'],
 ['아 진짜 까다로운 자식. 어디서 이런 자식이 나왔나?', '2'],
 ['수학과? 너 진짜 특이하다. 근데 수학과는 취업이

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [23]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 8,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [26]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [28]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-28-f37d0704c735>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/252 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.3074166774749756 train acc 0.046875
epoch 1 batch id 201 loss 1.4164674282073975 train acc 0.35401119402985076
epoch 1 train acc 0.37123842592592593


<ipython-input-28-f37d0704c735>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 0.49837680668473355


  0%|          | 0/252 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.8498015403747559 train acc 0.6875
epoch 5 batch id 201 loss 0.6051279306411743 train acc 0.7279228855721394
epoch 5 train acc 0.7327491181657848


  0%|          | 0/108 [00:00<?, ?it/s]

epoch 5 test acc 0.4999223690153568


In [29]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("fear가")
            elif np.argmax(logits) == 1:
                test_eval.append("surprise가")
            elif np.argmax(logits) == 2:
                test_eval.append("angry가")
            elif np.argmax(logits) == 3:
                test_eval.append("sad가")
            elif np.argmax(logits) == 4:
                test_eval.append("neutral가")
            elif np.argmax(logits) == 5:
                test_eval.append("happy가")
            elif np.argmax(logits) == 6:
                test_eval.append("dislike가")
            elif np.argmax(logits) == 7:
                test_eval.append("contempt가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [30]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 0


In [32]:
torch.save(model.state_dict(), '/content/kobert_model.pth')

In [33]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)
model.load_state_dict(torch.load('/content/kobert_model.pth'))

<All keys matched successfully>

In [34]:
total_data = BERTDataset(data_list, 0, 1, tok, vocab, max_len, True, False)
total_data_dataloader = torch.utils.data.DataLoader(total_data, batch_size = batch_size, num_workers = 5)

In [36]:
model.eval()

text_list = []
top1_emotions = []
top1_probabilities = []
top2_emotions = []
top2_probabilities = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(total_data_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    _, top2_indices = torch.topk(out, 2, dim=1)

    softmax = nn.Softmax(dim=1)
    probabilities = softmax(out)
    top1_probs = probabilities[range(probabilities.shape[0]), top2_indices[:, 0]]
    top2_probs = probabilities[range(probabilities.shape[0]), top2_indices[:, 1]]

    text_list.extend([text for text, _ in data_list[batch_id * batch_size:(batch_id + 1) * batch_size]])
    top1_emotions.extend(top2_indices[:, 0].tolist())
    top1_probabilities.extend(top1_probs.tolist())
    top2_emotions.extend(top2_indices[:, 1].tolist())
    top2_probabilities.extend(top2_probs.tolist())

result_data = {
    'Text': text_list,
    'Top1_Emotion': top1_emotions,
    'Top1_Probability': top1_probabilities,
    'Top2_Emotion': top2_emotions,
    'Top2_Probability': top2_probabilities
}

result_df = pd.DataFrame(result_data)

# CSV 파일로 저장
result_df.to_csv('/content/top_emotions_results.csv', index=False)


<ipython-input-36-eb0b3aeb6b12>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(total_data_dataloader)):


  0%|          | 0/359 [00:00<?, ?it/s]

In [37]:
data = pd.read_csv("/content/top_emotions_results.csv")

In [38]:
data

,Text,Top1_Emotion,Top1_Probability,Top2_Emotion,Top2_Probability
0,안 그래도 노인들은 무시 당하는 세상인데… 너무 부당한 일이 많잖아.,6,0.365460,3,0.330008
1,다음주에 엄마 생일이잖아. 잊은 건 아니지?,4,0.434068,3,0.296010
2,우와 그림 잘 그리시나봐요.,4,0.933640,5,0.030930
3,할 일이 너무 많은데 채점까지 우리한테 시키니까. 그게 본업도 아니고 한가한 것도 ...,6,0.520911,2,0.214776
4,"미안해, 누나. 이번에는 약속 진짜 지킬게. 책 새로 살 건 아니지?",3,0.835580,4,0.078248
...,...,...,...,...,...
22957,아까 문-이과 통합 사례에 대해 말씀하셨는데요,4,0.935024,5,0.032180
22958,나 지금 핸드폰으로 게임 할거야.,4,0.853327,6,0.058273
22959,아빠 나 좋은 소식이 있어.,5,0.908253,4,0.038876
22960,"어머, 나는 안 돼. 법에 걸려. 그런거 안 하셔도 된다고 말씀드려.",6,0.364361,0,0.347932
